In [ ]:
from google.colab import drive
import pandas as pd

from sklearn import svm

from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from sklearn import metrics

from statistics import mean

import random

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier

from sklearn.preprocessing import RobustScaler

from sklearn.inspection import permutation_importance

from sklearn.feature_selection import SelectKBest, chi2, f_classif

from sklearn.feature_selection import mutual_info_classif

import time

import numpy

from scipy.stats import pointbiserialr
from math import sqrt

import math

drive.mount('/content/drive')

pd.set_option("display.precision", 5)

data = pd.read_csv("/content/drive/My Drive/NSL-KDD/KDDTrain+_20Percent.txt", header=0, nrows=4000)

pd.options.mode.chained_assignment = None  # default='warn'

Mounted at /content/drive


In [ ]:
data = data.drop(['useless'], axis='columns')

In [ ]:
#convert categorical data to numerical data

dummies = pd.get_dummies(data.protocol_type)
dummies1 = pd.get_dummies(data.service)
dummies2 = pd.get_dummies(data.flag)

In [ ]:
#merge columns and drop categorical columns 
merged = pd.concat([data,dummies,dummies1,dummies2], axis = 'columns')
merged = merged.drop(['protocol_type','service','flag'], axis = 'columns')

In [ ]:
# convert class labels to normal and anomaly

for ind in merged.index:
  
  if merged['class'][ind] != 'normal': # conver categorical labels to numerical 

    merged['class'][ind] = 0  # if label is not normal change label to 0

  else:

    merged['class'][ind] = 1 # if label is normal chamge label to 1

In [ ]:
labels = merged['class'].tolist() # seperate labels

train_data = merged.drop(['class'], axis = 'columns') # drop label column for train data


In [ ]:
drops = []

columns = list(train_data)

for col in columns:

  if train_data[col].max() > 1:

    drops.append(col)

  elif train_data[col].sum() == 0:

    drops.append(col)

In [ ]:
train_data_dropped = train_data.drop(drops, axis = 'columns')

In [ ]:
train_data_dropped


,urgent,logged_in,root_shell,su_attempted,num_shells,is_guest_login,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,...,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF,SH
0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1.0,1.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0.2,0.2,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,1,0,0,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3996,0,0,0,0,0,0,0.0,0.0,1.0,1.0,...,0,0,0,1,0,0,0,0,0,0
3997,0,0,0,0,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3998,0,0,0,0,0,0,1.0,1.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
model = DecisionTreeClassifier()

model.fit(train_data_dropped, labels)
        
score = model.feature_importances_


In [ ]:

main_time = []

fitness_time = []

acc = []

f1_arr = [] 

recall_arr = []

precision_arr = []

for count in range(100):

  # variables

  parents = [] # most fit n chromosomes of the population

  score_list = []  # list of all scores 

  sorted_by_score = [] #sorted list for scores

  feature_subset_list = [] # list for feature_subsets

  drop_list = [] #list for features to be dropped

  # generate a random population of features  

  init_population =([[random.randint(0,1) for x in range(77)] for i in range(100)]) 

  population = init_population

  fitness_total_time = 0




  #MAIN LOOP

  main_start = time.process_time()

  while len(population) > 1:

    # CALCULATE SCORE

    score_list = []

    fitness_start = time.process_time()

    for i in range(len(population)):

      f_score = 0

      for j in range(len(population[i])):

        if population[i][j] == 1:

          f_score += score[j]

      score_list.append(f_score)

    fitness_end = time.process_time() - fitness_start 

    fitness_total_time += fitness_end 

    #SORT POPULATION ACCORDING TO SCORES

    zipped = zip(score_list,population)

    sorted_by_score = [x for _, x in sorted(zipped)]



    #PARENT SELECTION

    parents = sorted_by_score[int(3*len(sorted_by_score)/4):]

    if len(parents) % 2 != 0:

      del parents[0]

    #CROSSOVER

    cross_point = random.randint(0,75)

    l = len(parents)-1

    for k in range(int(len(parents)/2)):

      parents.append(parents[k][0:cross_point] + parents[l][cross_point:75])

      parents.append(parents[l][0:cross_point] + parents[k][cross_point:75])

      l = l-1

    population = parents


    #MUTATION

    for n in range(0,len(population)):

      mutation_point = random.randint(0,74)

      if(population[n][mutation_point]) == 0:

        population[n][mutation_point] = 1

      else:

        population[n][mutation_point] = 0



  main_end = time.process_time() - main_start

  main_time.append(main_end)

  fitness_time.append(fitness_total_time)

  #TESTING THE FITTEST


  dropper = []

  for o in range(len(sorted_by_score[3])):

    if sorted_by_score[3][o] == 0:
      
      dropper.append(train_data_dropped.columns[o])

  subs = train_data_dropped.drop(dropper, axis = 'columns')

  fitter = subs.values.tolist()

  x_train, x_test, y_train, y_test = train_test_split(fitter, labels, test_size = 0.2, random_state = 0)

  the_model = svm.SVC()

  the_model.fit(x_train,y_train)

  the_pred = the_model.predict(x_test)

  the_score = metrics.accuracy_score(y_test, the_pred)

  f1 = metrics.f1_score(y_test,the_pred, average = 'binary')

  f1_arr.append(f1)

  recall = metrics.recall_score(y_test, the_pred, average = 'binary')

  recall_arr.append(recall)

  precision = metrics.precision_score(y_test, the_pred, average = 'binary')

  precision_arr.append(precision)

  acc.append(the_score)


In [ ]:
print("Average accuracy: " , mean(acc))

print("Average main time: " , mean(main_time))

print("Average f1 score: " , mean(f1_arr))

print("Average fitness time: " , mean(fitness_time))

print("Average recall score: " , mean(recall_arr))

print("Average precision score: " , mean(precision_arr))

overall_mean = []



Average accuracy:  0.97625
Average main time:  0.004379818349999951
Average f1 score:  0.9773701987874487
Average fitness time:  0.004000716899999963
Average recall score:  0.9934382566585956
Average precision score:  0.9618187023355471


In [ ]:
overall_mean.append(mean(acc))
overall_mean.append(mean(main_time))
overall_mean.append(mean(f1_arr))
overall_mean.append(mean(fitness_time))
overall_mean.append(mean(recall_arr))
overall_mean.append(mean(precision_arr))

overall_mean

[0.97625,
 0.004379818349999951,
 0.9773701987874487,
 0.004000716899999963,
 0.9934382566585956,
 0.9618187023355471]

In [ ]:
print("Standard deviation of accuracy: " , numpy.std(acc))
print("Standard deviation of main time: " , numpy.std(main_time))
print("Standard deviation of f1 scores: " , numpy.std(f1_arr))
print("Standard deviation of fitness time: " , numpy.std(fitness_time))
print("Standard deviation of recall: " , numpy.std(recall_arr))
print("Standard deviation of precision: " , numpy.std(precision_arr))


overall_stdev = []

Standard deviation of accuracy:  0.0011180339887499108
Standard deviation of main time:  0.0014552400159220638
Standard deviation of f1 scores:  0.001052802741123315
Standard deviation of fitness time:  0.0014189814540251677
Standard deviation of recall:  0.00165270866310714
Standard deviation of precision:  0.002120930175435204


In [ ]:
overall_stdev.append(numpy.std(acc))
overall_stdev.append(numpy.std(main_time))
overall_stdev.append(numpy.std(f1_arr))
overall_stdev.append(numpy.std(fitness_time))
overall_stdev.append(numpy.std(recall_arr))
overall_stdev.append(numpy.std(precision_arr))

overall_stdev

[0.0011180339887499108,
 0.0014552400159220638,
 0.001052802741123315,
 0.0014189814540251677,
 0.00165270866310714,
 0.002120930175435204]

In [ ]:
with open('/content/drive/My Drive/NSL-KDD/fga_all_metrics/fga_avg_4000.txt', 'w') as f:
    numpy.savetxt(f, numpy.array(overall_mean))

In [ ]:
with open('/content/drive/My Drive/NSL-KDD/fga_all_metrics/fga_stdev_4000.txt', 'w') as f:
    numpy.savetxt(f, overall_stdev)

In [ ]:
#np_main = numpy.array(main_time)

#np_fitness = numpy.array(fitness_time)

#np_acc = numpy.array(acc)

#np_f1_arr = numpy.array(f1_arr)

#np_precision_arr = numpy.array(precision_arr)

#np_recall_arr = numpy.array(recall_arr)

In [ ]:
#with open('/content/drive/My Drive/NSL-KDD/time_results_fga/main_time_500.txt', 'w') as f:
 #   numpy.savetxt(f, np_main)

In [ ]:
#with open('/content/drive/My Drive/NSL-KDD/time_results_fga/fitness_time_500.txt', 'w') as f:
 #   numpy.savetxt(f, np_fitness)

In [ ]:
#with open('/content/drive/My Drive/NSL-KDD/accuracy_results_fga/accuracy_500.txt', 'w') as f:
 #   numpy.savetxt(f, np_acc)